In [1]:
import os

import numpy as np
from dipy.io.streamline import load_tractogram
from dipy.segment.clustering import QuickBundles
from dipy.io.pickles import save_pickle
from dipy.data import get_fnames

/home/teitxe/projects/tractoencoder_gsoc/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load FiberCup fibers (training, plausibles)

In [2]:
# Load tractogram
fname = "../data/fibercup/fibercup_advanced_filtering_no_ushapes/ae_input_std_endpoints/train/fibercup_Simulated_prob_tracking_minL10_resampled256_plausibles_std_endpoints_train.trk"
tractogram = load_tractogram(fname, "same", bbox_valid_check=False)
streamlines = tractogram.streamlines

### Use QuickBundles to segment the bundles in FiberCup

In [3]:
qb = QuickBundles(threshold=10.)
clusters = qb.cluster(streamlines)
bundles = [np.array(bundle) for bundle in clusters]

print("Nb. clusters:", len(clusters))
print("Cluster sizes:", [len(cluster) for cluster in clusters])
print("Small clusters:", clusters < 10)

Nb. clusters: 7
Cluster sizes: [341, 592, 296, 256, 511, 448, 668]
Small clusters: [False False False False False False False]


### Save each bundle into a .trk file

In [5]:
import tractoencoder_gsoc.utils as utils
# Reference anatomy
ref_anat = "../data/fibercup/Simulated_FiberCup.nii.gz"
output_path = "../data/fibercup/fibercup_advanced_filtering_no_ushapes/ae_input_std_endpoints/train/quickbundles"
os.makedirs(output_path, exist_ok=True)
for i, bundle in enumerate(bundles):
    bundle_name = f"bundle_{i}"
    bundle_trk_path = os.path.join(output_path, f"{bundle_name}.trk")
    utils.save_tractogram(streamlines=bundle,
                          tractogram_fname=bundle_trk_path,
                          img_fname=ref_anat)

### Load tractograms to check if saving was successfull

In [ ]:
bundle_fname = "../data/fibercup/fibercup_advanced_filtering_no_ushapes/ae_input_std_endpoints/train/quickbundles/bundle_0.trk"
utils.prepare_tensor_from_file(bundle_trk_path, img_fname=ref_anat)